<h2 style="text-align: center;"><strong>Steam Challenge for Datatonic</strong></h2>
<h3 style="text-align: center;">Steven Jordan</h3>
<p><strong>Exercise 1: Data Engineering</strong></p>
<p style="padding-left: 30px;">1. Install and run PySpark</p>
<p style="padding-left: 30px;">PySpark was run in the cloud using DataProc as part of Google Cloud Platform. A cluster was set up and connected to Jupyter.</p>

In [1]:
# Install PySpark libraries and initiate Spark
!pip install --upgrade  pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, col, avg
spark = SparkSession.builder.getOrCreate()

  Found existing installation: pyspark 2.3.4
    Can't uninstall 'pyspark'. No files were found to uninstall.


<p style="padding-left: 30px;">2. Load .csv for Player_Summaries, Games_Publishers, Games_Genres, Games_Developers, Games_1 into PySpark dataframes.</p>

In [2]:
# Read the Player Summary csv files and combine into a single dataframe
path = "gs://dataproc-d5da8056-80df-436a-8ab5-db077106cb06-europe-west6/notebooks/"
player_summaries0_df = spark.read.csv(path + "Player_Summaries-000000000000.csv", header=True)
player_summaries1_df = spark.read.csv(path + "Player_Summaries-000000000001.csv", header=True)
player_summaries2_df = spark.read.csv(path + "Player_Summaries-000000000002.csv", header=True)
player_summaries3_df = spark.read.csv(path + "Player_Summaries-000000000003.csv", header=True)
player_summaries4_df = spark.read.csv(path + "Player_Summaries-000000000004.csv", header=True)
player_summaries5_df = spark.read.csv(path + "Player_Summaries-000000000005.csv", header=True)
player_summaries_df = player_summaries0_df.union(player_summaries1_df.union(player_summaries2_df.union(player_summaries3_df.union(player_summaries4_df.union(player_summaries5_df)))))

# Read the Games_* csv files and combine the Games_1 files into single dataframes
games_publishers_df = spark.read.csv(path + "Games_Publishers.csv", header=True)
games_developers_df = spark.read.csv(path + "Games_Developers.csv", header=True)
games_genres_df = spark.read.csv(path + "Games_Genres.csv", header=True)

games_10_df = spark.read.csv(path + "Games_1-000000000000.csv", header=True)
games_11_df = spark.read.csv(path + "Games_1-000000000001.csv", header=True)
games_1_df = games_10_df.union(games_11_df)

## The instructions did not state to load the Games_2 files - but the following instruction says to include all Games_, 
## so I assume it was ommitted by typo
games_20_df = spark.read.csv(path + "Games_2-000000000000.csv", header=True)
games_21_df = spark.read.csv(path + "Games_2-000000000001.csv", header=True)
games_22_df = spark.read.csv(path + "Games_2-000000000002.csv", header=True)
games_2_df = games_20_df.union(games_21_df.union(games_22_df))

games_full_df = games_1_df.union(games_2_df)

<p style="padding-left: 30px;">3. Join all 'Games_' tables into one dataframe.</p>

In [3]:
games_join = games_full_df.join(games_publishers_df, on = ['appid'], how = 'full').join(games_developers_df, on = ['appid'], how = 'full').join(games_genres_df, on = ['appid'], how = 'full')

In [4]:
games_join.show(10)

+------+-----------------+---------------+----------------+--------------------+---------------+--------------+--------+
| appid|          steamid|playtime_2weeks|playtime_forever|       dateretrieved|      Publisher|     Developer|   Genre|
+------+-----------------+---------------+----------------+--------------------+---------------+--------------+--------+
|108231|             null|           null|            null|                null| Days of Wonder|Days of Wonder|   Indie|
|108231|             null|           null|            null|                null| Days of Wonder|Days of Wonder|  Casual|
|108231|             null|           null|            null|                null| Days of Wonder|Days of Wonder|Strategy|
|108800|76561197973718054|           null|            1348|2013-05-14 07:52:...|Electronic Arts|Crytek Studios|  Action|
|108800|76561198077027953|            213|             969|2013-09-27 11:45:...|Electronic Arts|Crytek Studios|  Action|
|108800|76561197970336122|      

<p style="padding-left: 30px;">4. Count the number of games per 'publisher' and per 'genre.'</p>

In [5]:
games_publishers_df.groupBy('Publisher').count().orderBy('count', ascending = 0).show(truncate=False)

+--------------------------------------+-----+
|Publisher                             |count|
+--------------------------------------+-----+
|null                                  |2627 |
|Ubisoft                               |384  |
|SEGA                                  |349  |
|Dovetail Games - Trains               |279  |
|Paradox Interactive                   |246  |
|Disney Interactive                    |226  |
|Feral Interactive (Mac)               |221  |
|Activision                            |221  |
|Degica                                |190  |
|Nordic Games                          |164  |
|Square Enix                           |153  |
|KISS ltd                              |145  |
|Feral Interactive (Linux)             |143  |
|Wizards of the Coast LLC              |135  |
|Strategy First                        |134  |
|2K Games                              |120  |
|Capcom                                |115  |
|Warner Bros. Interactive Entertainment|108  |
|Kalypso Medi

In [6]:
games_genres_df.groupBy('Genre').count().orderBy('count', ascending = 0).show(truncate=False)

+---------------------+-----+
|Genre                |count|
+---------------------+-----+
|Indie                |7982 |
|Action               |7126 |
|Adventure            |4517 |
|Strategy             |3953 |
|Casual               |3939 |
|Simulation           |3870 |
|RPG                  |3147 |
|Free to Play         |1172 |
|Early Access         |901  |
|Massively Multiplayer|748  |
|Racing               |619  |
|Sports               |604  |
|Design & Illustration|240  |
|Utilities            |207  |
|Web Publishing       |146  |
|Animation & Modeling |112  |
|Audio Production     |112  |
|Software Training    |82   |
|Education            |73   |
|Video Production     |69   |
+---------------------+-----+
only showing top 20 rows



<p style="padding-left: 30px;">5. Find day and hour when most new accounts were created (based on Player_Summaries table)</p>

In [7]:
player_summaries_df.groupBy(date_format('timecreated', 'yyyy-MM-dd HH').alias('hour')).count().orderBy('count', ascending = 0).show(5)

+-------------+------+
|         hour| count|
+-------------+------+
|         null|252717|
|2013-03-02 10|  1782|
|2012-12-25 10|  1448|
|2012-12-25 08|  1355|
|2012-12-25 11|  1159|
+-------------+------+
only showing top 5 rows



<p style="padding-left: 30px;">The most accounts (with non-null values) were created on March 2nd, 2013 at 10 AM.</p>

<p><strong>Exercise 2: Analytics</strong></p>
<p style="padding-left: 30px;">Case Instructions: "Your client is a mental health expert from an NGO who is interested in understanding more about gaming and the potentially addictive effect it can have on some individuals. You are meeting the client in a few days and they would like you to extract and present insights from the Steam dataset to help them in their research."</p>

I would ideally obtain my client's benchmarks that indicate an addiction to video games, but it was not provided. I researched how many hours of video gaming per day/week would be considered an addiction, but different metrics are often cited (and none is provided in the WHO's classification of gaming disorders). So I settled on 50 hours per week as a middle-ground between many of the figures. Using this, I used the data to answer the following questions (visualizations are provided via Tableau).

1. What percentage of Steam users have an "addiction-level" playtime?

In [8]:
# Open the App ID file to obtain additional information bout the games
app_id_df = spark.read.csv(path + "App_ID_Info.csv", header=True)

# Remove games that have never been played, join the App ID dataframe, and remove unnecessary columns
games_played = games_join.where(col('playtime_forever').isNotNull())
games_played = games_played.join(app_id_df, on = ['appid'], how = 'left')
games_played = games_played.drop(*['dateretrieved', 'Type', 'Release_Date', 'Rating', 'Required_Age', 'Is_Multiplayer'])

In [9]:
games_played.show(5)

+-----+-----------------+---------------+----------------+-------------------+--------------------+----------+--------------------+-----+
|appid|          steamid|playtime_2weeks|playtime_forever|          Publisher|           Developer|     Genre|               Title|Price|
+-----+-----------------+---------------+----------------+-------------------+--------------------+----------+--------------------+-----+
|55230|76561197972368092|           null|            2919|        Deep Silver|            Volition|    Action|Saints Row: The T...|14.99|
|42910|76561197972228702|           null|              18|Paradox Interactive|Arrowhead Game St...|    Action|             Magicka| 9.99|
|42910|76561197972228702|           null|              18|Paradox Interactive|Arrowhead Game St...|       RPG|             Magicka| 9.99|
|33900|76561198058121020|           null|              13|Bohemia Interactive| Bohemia Interactive|Simulation|              Arma 2|12.99|
|33900|76561198058121020|         

In [10]:
# Convert the numeric features to int/float
from pyspark.sql.types import IntegerType, FloatType
games_played = games_played.withColumn('playtime_2weeks', games_played['playtime_2weeks'].cast(IntegerType()))
games_played = games_played.withColumn('playtime_forever', games_played['playtime_forever'].cast(IntegerType()))
games_played = games_played.withColumn('Price', games_played['Price'].cast(FloatType()))

In [11]:
# Create a dataframe of the players with an addiction-level of playtime in the last two weeks (100 hours or 6000 minutes)
addicts_df = games_played.groupBy('steamid').sum('playtime_2weeks').where(col('sum(playtime_2weeks)') > 6000)
addicts_df.show(5)

+-----------------+--------------------+
|          steamid|sum(playtime_2weeks)|
+-----------------+--------------------+
|76561197979880077|               19835|
|76561198017077515|               45219|
|76561197996880035|               29113|
|76561198043166176|               10340|
|76561197971697538|               18111|
+-----------------+--------------------+
only showing top 5 rows



In [12]:
# Create a dataframe of the total number of "active" gamers that have any playtime in the last two weeks
active_players_df = games_played.groupBy('steamid').sum('playtime_2weeks').where(col('sum(playtime_2weeks)') > 0)


In [13]:
# Calculate addiction rate
addiction_rate = addicts_df.count()/active_players_df.count()
print(addiction_rate)

0.24729512684796495


2. Identify the most popular games, developers, and genres for the addicted-level players (by number of downloads)

In [14]:
# Create a dataframe with the game details, but that only includes the "addicted-level players"
addicts_games_df = addicts_df.join(games_played, on = ['steamid'], how = 'left')

# Identify the most popular game titles for addicted-level players
most_pop_games = addicts_games_df.select(['steamid', 'Title']).dropDuplicates()
most_pop_games.groupBy('Title').count().orderBy('count', ascending = 0).show(11, truncate = False)

# Export to CSV for Tableau
most_pop_games.groupBy('Title').count().orderBy('count', ascending = 0).repartition(1).write.csv(path + 'most_pop_games.csv')

+--------------------------------+------+
|Title                           |count |
+--------------------------------+------+
|null                            |122736|
|Dota 2                          |86214 |
|Team Fortress 2                 |68351 |
|Left 4 Dead 2                   |63188 |
|ORION: Prelude                  |47755 |
|Counter-Strike: Global Offensive|47541 |
|Borderlands 2                   |45277 |
|The Elder Scrolls V: Skyrim     |44572 |
|Counter-Strike: Source          |41766 |
|Contagion                       |41597 |
|Sid Meier's Civilization® V     |41345 |
+--------------------------------+------+
only showing top 11 rows



In [15]:
# Identify the most popular developers of games downloaded by addicted-level players
most_pop_devs = addicts_games_df.select(['steamid', 'Developer']).dropDuplicates()
most_pop_devs.groupBy('Developer').count().orderBy('count', ascending = 0).show(11, truncate = False)

# Export to CSV for Tableau
most_pop_devs.groupBy('Developer').count().orderBy('count', ascending = 0).repartition(1).write.csv(path + 'most_pop_devs.csv')

+-----------------------+------+
|Developer              |count |
+-----------------------+------+
|null                   |123541|
|Valve                  |121205|
|Feral Interactive (Mac)|63191 |
|Infinity Ward          |54492 |
|Gearbox Software       |54291 |
|Firaxis Games          |49169 |
|Aspyr (Mac)            |48560 |
|Bethesda Game Studios  |48014 |
|Spiral Game Studios    |47755 |
|Aspyr (Mac & Linux)    |45277 |
|Facepunch Studios      |45147 |
+-----------------------+------+
only showing top 11 rows



In [16]:
# Identify the most popular genres of games downloaded by addicted-level players
most_pop_genre = addicts_games_df.select(['steamid', 'Genre']).dropDuplicates()
most_pop_genre.groupBy('Genre').count().orderBy('count', ascending = 0).show(11, truncate = False)

# Export to CSV for Tableau
most_pop_genre.groupBy('Genre').count().orderBy('count', ascending = 0).repartition(1).write.csv(path + 'most_pop_genre.csv')

+---------------------+------+
|Genre                |count |
+---------------------+------+
|Action               |133740|
|Strategy             |129858|
|Indie                |122848|
|null                 |122791|
|Free to Play         |119091|
|Adventure            |110148|
|RPG                  |101159|
|Simulation           |86835 |
|Casual               |86315 |
|Massively Multiplayer|72717 |
|Early Access         |65014 |
+---------------------+------+
only showing top 11 rows



3. Identify the most popular game titles for addicted-level players (by overall playtime)

In [17]:
# Rank the titles by play time
most_played_games = addicts_games_df.select(['playtime_forever', 'Title']).dropDuplicates()
most_played_games.groupBy('Title').sum('playtime_forever').orderBy('sum(playtime_forever)', ascending = 0).show(11, truncate = False)

# Export to CSV for Tableau
most_played_games.groupBy('Title').sum('playtime_forever').orderBy('sum(playtime_forever)', ascending = 0).repartition(1).write.csv(path + 'most_played_games_pt.csv')

+--------------------------------+---------------------+
|Title                           |sum(playtime_forever)|
+--------------------------------+---------------------+
|Dota 2                          |5391727528           |
|Team Fortress 2                 |1197885641           |
|Sid Meier's Civilization® V     |777257322            |
|null                            |754706756            |
|Counter-Strike                  |569676280            |
|Counter-Strike: Source          |549369386            |
|Garry's Mod                     |483423927            |
|The Elder Scrolls V: Skyrim     |386992154            |
|Counter-Strike: Global Offensive|354250698            |
|Call of Duty®: Modern Warfare® 2|277854837            |
|Call of Duty®: Modern Warfare® 3|226900035            |
+--------------------------------+---------------------+
only showing top 11 rows



4. Calculate how much money is spent in Steam by addicted-level players.

In [18]:
# Rank each player by overall spend, removing duplicates
most_money_spent = addicts_games_df.select(['steamid', 'Title', 'Price']).dropDuplicates()
most_money_spent.groupBy('steamid').sum('Price').orderBy('sum(Price)', ascending = 0).show(truncate = False)

# Export to CSV for Tableau
most_money_spent.groupBy('steamid').sum('Price').orderBy('sum(Price)', ascending = 0).repartition(1).write.csv(path + 'most_money_spent.csv')

+-----------------+------------------+
|steamid          |sum(Price)        |
+-----------------+------------------+
|76561197986448970|32302.839499235153|
|76561197971880992|29075.59961295128 |
|76561197977129171|25774.79963350296 |
|76561197972029264|23950.169701576233|
|76561198013213702|23003.319592237473|
|76561197970335892|22973.59959578514 |
|76561197971066622|22494.21960043907 |
|76561197970432492|22191.15969467163 |
|76561197993174473|22184.2297000885  |
|76561197970322620|21834.309688329697|
|76561197970408576|21600.409688711166|
|76561198024662414|21556.259591579437|
|76561198008037678|21492.76966881752 |
|76561198033882875|21311.44964647293 |
|76561197979750959|20412.259642362595|
|76561197988967716|20049.37964630127 |
|76561198004780374|19934.09962081909 |
|76561198050036517|19899.31965470314 |
|76561198053167763|18797.699720144272|
|76561198014782362|18626.929695367813|
+-----------------+------------------+
only showing top 20 rows



In [19]:
# Find the average amount spent by the addicted-level players
average_spent = most_money_spent.groupBy('steamid').sum('Price').select(avg(col('sum(Price)')))
average_spent.show()

+----------------+
| avg(sum(Price))|
+----------------+
|692.323092390373|
+----------------+



5. Find the country locations of the most addicted-level players

In [20]:
# Join the player summary dataframes to the addicted-level players
addicts_loc_df = addicts_df.join(player_summaries_df, on = ['steamid'], how = 'left')

# Export to CSV for Tableau
addicts_loc_df.groupBy('loccountrycode').count().orderBy('count', ascending=0).repartition(1).write.csv(path + 'addicts_loc_df.csv')